In [13]:
%reload_ext jupyter_black

**Key takeaways from the dataset**

* Some articles have no image
* Some customers don't buy anything
* The complete transaction data has 31 788 325 rows, just short of 32 million (!!)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import Union, Tuple
from types import NoneType
import random, shutil, os, itertools, black, jupyter_black

jupyter_black.load()

## Sampling methods

We need to be able to pull out realistic samples of the dataset. To do this, we first sample $n$ customers at random and include every transaction that they have done - these are the positive labels. In addition, we want to obtain additional transactions that are not related to the customers in the sample, working as a negative label. We implement this by saying that $k$% of the data are true labels, defaulting $k=10$%. Lastly, we pull out the article IDs in all the transactions and obtain the images for said article.

In [ ]:
def naive_csv_sampler(
    csv_path: str,
    sample_size: int,
    num_records: int | NoneType = None,
    header: str | NoneType = "infer",
) -> pd.DataFrame:
    """Read samples of rows from csv file

    Args:
        csv_path (str): Path to file including file extensions
        sample_size (int): Number of rows to sample
        num_records (int | NoneType, optional): Total records in file, defaults to None. If None, the file will be scanned (costly)
        header (str | NoneType, optional): 'header'-parameter for pandas, defaults to 'infer'. Set to None if file has no header.

    Returns:
        pd.DataFrame: Dataframe with sampled entries (and potentially header)
    """
    if num_records is None:
        num_records = newlines_in_csv(csv_path)
    indices_skip = sorted(
        random.sample(range(1, num_records + 1), num_records - sample_size)
    )
    return pd.read_csv(csv_path, skiprows=indices_skip, header=header)


def newlines_in_csv(csv_path: str, chunk_size: int = 1024) -> int:
    """Counts number of newlines in csv file without loading entire file to memory.
    The number of newlines is the same as number of rows assuming,
        * EITHER csv has a header and last entry does not end with newline
        * OR csv does not have a header, but last entry ends with newline
        * ALWAYS data does not have any nested newline madness
    Originally from orlp, https://stackoverflow.com/a/64744699

    Args:
        csv_path (str): Path of csv file
        chunk_size (int, optional): How many KB to process at at a time. Defaults to 1024 = 1 MB.

    Returns:
        int: Number of newlines
    """
    chunk = chunk_size**2
    f = np.memmap(csv_path)
    number_newlines = sum(
        np.sum(f[i : i + chunk] == ord("\n")) for i in range(0, len(f), chunk)
    )
    del f
    return number_newlines


In [16]:
def copy_img_from_article(df: pd.DataFrame, outpath):
    for id in df['article_id']:
        id0 = "0" + str(id)
        img_path = f"./dataset/images/{id0[:3]}/{id0}.jpg"
        if not os.path.isfile(img_path):
            continue # ID has no image (happens for some cases)
        out_dir = f"./{outpath}/images/{id0[:3]}/"
        if not os.path.isdir(out_dir):
            os.makedirs(out_dir)
            shutil.copy(img_path, out_dir)
copy_img_from_article(df_art, "dataset_sample")

## Data loading and preprocessing

In [5]:
# Cleaning up the datasets
from typing import Iterable


def load_min_data(filename: str | Iterable):
    dfs = []
    if isinstance(filename, str):
        filename = [filename]
    for fn in filename:
        df = pd.read_csv(fn)
        # All min-datasets have an index column which has to be dropped:
        dfs.append(df.drop(df.columns[0], axis=1))
    return dfs

def clean_customer_data(df):
    # df = df.drop("FN", axis=1) # I they're not exactly equal
    df.loc[
        ~df["fashion_news_frequency"].isin(["Regularly", "Monthly"]),
        "fashion_news_frequency",
    ] = "None"
    return df


Number of rows in complete transactions csv:

In [33]:
newlines_in_csv("dataset/transactions_train.csv")

31788325

<hr>

### Trying out user-user collaborative filtering

In [44]:
"""Psudeocode
* Create customer profiles for all customers in (sampled) dataset
    * i.e. each customer ID has a vector r_ID whose elements represent items purchased
* Compute Jaccard similarity between all r_IDs, independent of position
* For a given customer x, choose the k customers closest to x
* For an article i, wether or not to recommend is based on the recommendation score
    r(x, i) = mean( [rel(y, i) for y in top k] )
"""


def position_indep_jaccard(x: list | set, y: list | set) -> float:
    # Position-independent jaccard-similarity
    x, y = set(x), set(y)
    return len(x.intersection(y)) / len(x.union(y))



def find_customer_similarity(
    df_customer: pd.DataFrame, df_transactions: pd.DataFrame
) -> Tuple[pd.DataFrame, dict]:
    articles_dict = {}
    for cust_ID in df_customer["customer_id"]:
        articles_dict[cust_ID] = df_transactions["article_id"][
            df_transactions["customer_id"] == cust_ID
        ].to_list()
        # Pop customers without purchase history
        if len(articles_dict[cust_ID]) == 0:
            articles_dict.pop(cust_ID)
    num_customers = len(df_customer)
    print(f"{num_customers = }")
    similarity_matrix = np.zeros((num_customers, num_customers))
    # Iterate over customers:
    for r, cust in enumerate(articles_dict.keys()):
        for c, second in enumerate(articles_dict.keys()):
            sim = position_indep_jaccard(articles_dict[cust], articles_dict[second])
            similarity_matrix[r, c] = sim

    return (
        pd.DataFrame(
            similarity_matrix, index=articles_dict.keys(), columns=articles_dict.keys()
        ),
        articles_dict,
    )


def get_recommendation(
    similarity_matrix: pd.DataFrame,
    articles_dict: dict,
    customer_ID: str,
    article_ID: int,
    k: int,
) -> float:
    """Produce recommendation score of an item based on its k closest customer behaviors

    Args:
        similarity_matrix (pd.DataFrame): nxn matrix of similarities between customers
        articles_dict (dict): Dictionary of customer purchases on form {customer_id: [item1, item2, ...]}
        customer_ID (str): The customer the recommendation score is based on
        article_ID (int): The article the score is based on
        k (int): How many (closest) customer-neighbors to include in computation.

    Returns:
        float: Measure of how well the item would fit the customer in question, between [0,1]
    """
    # The k most similar customers IDs:
    closest_customers = (
        similarity_matrix[customer_ID].sort_values(ascending=False)[:k].index
    )
    return (
        sum(1 if article_ID in articles_dict[cust] else 0 for cust in closest_customers)
        / k
    )

In [27]:
# Load sample data
df_cust = pd.read_csv("dataset_sample/customer_min.csv")
df_tr = pd.read_csv("dataset_sample/transactions_min.csv")
df_art = pd.read_csv("dataset_sample/articles_min.csv")
sim_matr, art_dict = find_customer_similarity(df_cust, df_tr)

num_customers = 200


In [37]:
def get_n_recommendations(
    n: int,
    similarity_matrix: pd.DataFrame,
    articles_dict: dict,
    customer_ID: str,
    k: int,
    ignore_purchased: bool = True,
) -> list:
    """Get the n 'best' recommended items for a specific customer ID

    Args:
        n (int): How many items to recommend
        similarity_matrix (pd.DataFrame): Customer similarity matrix
        articles_dict (dict): Dictionary of customer purchases on form {customer_id: [item1, item2, ...]}
        customer_ID (str): _description_
        k (int): _description_
        ignore_purchased (bool, optional): _description_. Defaults to True.

    Returns:
        list: _description_
    """
    # Get rec. score for all cases and choose n with highest score
    # ignore_purchased to ignore those articles customer has already bought
    blacklisted_articles = (
        set(articles_dict[customer_ID]) if ignore_purchased else set()
    )
    art_IDs = set(itertools.chain(*articles_dict.values())) - blacklisted_articles
    score_dict = {
        art_ID: get_recommendation(
            similarity_matrix, articles_dict, customer_ID, art_ID, k
        )
        for art_ID in art_IDs
    }
    n_best_items = {
        k: v for k, v in sorted(score_dict.items(), key=lambda el: el[1], reverse=True)
    }
    # Return entire dict for debug purposes, but otherwise just the article IDs (not scores)
    return list(itertools.islice(n_best_items.items(), n))
    return list(n_best_items.keys())[:n]

In [43]:
get_n_recommendations(
    n=5,
    similarity_matrix=sim_matr,
    articles_dict=art_dict,
    customer_ID="008068b49b6bdd622ed406e30c8603270770174ebf300dbac0f5beac522921e0",
    k=5,
)

[(770851001, 0.2),
 (806388003, 0.2),
 (615154002, 0.2),
 (830702001, 0.2),
 (677561001, 0.2)]

In [26]:

get_recommendation(
    similarity_matrix=sim_matr,
    articles_dict=art_dict,
    customer_ID='008068b49b6bdd622ed406e30c8603270770174ebf300dbac0f5beac522921e0',
    article_ID=556255001,
    k=5
)


0.4

In this case, two of the $k$ closest customers (including the customer itself) has bought the article in question. Thus we get a score of $\frac25=0.4$